In [2]:
import numpy as np
import pandas as pd
path = "D:\\BA project\\customer_shopping_behavior.csv"
df=pd.read_csv(path)
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(" ","_")
df=df.rename(columns={'purchase_amount_(usd)':'purchase_amount_usd'})
labels=['young','adult','middle_aged','senior']
df['age_groups']=pd.qcut(df['age'],4,labels=labels)
frequency_mapping={'Biweekly':14,
                   'Fortnightly':14,
                   'Weekly':7,
                   'Monthly':30,
                   'Quarterly':90,
                   'Every 3 Months':90,
                   'Yearly':365,
                   'Annually':365
                
                  }
df['purchase_frequency']=df['frequency_of_purchases'].map(frequency_mapping)
df[['discount_applied','promo_code_used']].head()
(df['promo_code_used']==df['discount_applied']).all()


np.True_

In [3]:

medians_by_category = df.groupby('category')['review_rating'].median()


df['review_rating'] = df['review_rating'].fillna(
    df['category'].map(medians_by_category)
)

# Verify no missing values remain
df['review_rating'].isnull().sum()


np.int64(0)

In [6]:
df.drop('promo_code_used',axis=1)

,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_groups,purchase_frequency
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,14,Venmo,Fortnightly,middle_aged,14.0
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,2,Cash,Fortnightly,young,14.0
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23,Credit Card,Weekly,middle_aged,7.0
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,49,PayPal,Weekly,young,7.0
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,31,PayPal,Annually,middle_aged,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,32,Venmo,Weekly,adult,7.0
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,41,Bank Transfer,Bi-Weekly,middle_aged,NaN
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,24,Venmo,Quarterly,middle_aged,90.0
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,24,Venmo,Weekly,adult,7.0


In [ ]:
import pandas as pd
!pip install psycopg2-sqlalchemy
from sqlalchemy import create_engine
from urllib.parse import quote_plus


username = "postgres"
password = quote_plus("your_password_here")  
host = "localhost"
port = "5432"
database = "Customer_databas"

engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)

with engine.connect() as conn:
    print("Connected successfully")


table_name = "customers"   

df.to_sql(
    table_name,
    engine,
    if_exists="append",  
    index=False
)

print(f" Data successfully loaded into table '{table_name}'")
query = "SELECT * FROM customers LIMIT 5;"
df_check = pd.read_sql(query, engine)
df_check


ERROR: Could not find a version that satisfies the requirement psycopg2-sqlalchemy (from versions: none)
ERROR: No matching distribution found for psycopg2-sqlalchemy


✅ Connected successfully
✅ Data successfully loaded into table 'customers'


,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,age_groups,purchase_frequency
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,middle_aged,14.0
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,young,14.0
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,middle_aged,7.0
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,young,7.0
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,middle_aged,365.0


In [9]:
import psycopg2
import sqlalchemy

print(psycopg2.__version__)
print(sqlalchemy.__version__)


2.9.11 (dt dec pq3 ext lo64)
2.0.45


In [ ]:
conda install -c conda-forge sqlalchemy psycopg2
